In [3]:
import streamlit as st

with st.echo():
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager

    @st.experimental_singleton
    def get_driver():
        return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    options = Options()
    options.add_argument('--disable-gpu')
    options.add_argument('--headless')

    driver = get_driver()
    driver.get("http://example.com")

    st.code(driver.page_source)

2024-02-01 22:04:18.154 
  command:

    streamlit run /Users/han.med/Coding/zillow-dealfinder/v_env/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


ModuleNotFoundError: No module named 'webdriver_manager'